In [2]:
import os
from pyspark import SparkConf, SparkContext
import pandas as pd

In [3]:
!hostname
!cat /etc/hosts

KRAFTONui-MacBookPro.local
##
# Host Database
#
# localhost is used to configure the loopback interface
# when the system is booting.  Do not change this entry.
##
127.0.0.1	localhost
127.0.0.1 KRAFTONui-MacBookPro.local
255.255.255.255	broadcasthost
::1             localhost
# Added by Docker Desktop
# To allow the same kube context to work on the host and the container:
127.0.0.1 kubernetes.docker.internal
# End of section


In [4]:
conf = SparkConf().setMaster("local").setAppName("rdd-test")
sc = SparkContext(conf=conf)

22/03/11 12:45:22 WARN Utils: Your hostname, KRAFTONui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.17 instead (on interface en0)
22/03/11 12:45:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/11 12:45:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
sc

<SparkContext master=local appName=rdd-test>

In [6]:
type(sc)

pyspark.context.SparkContext

# RDD 만들기

In [7]:
trip_files = "/Users/krafton/project/personal/spark-airflow-hands-on/data/trips/yellow_tripdata_2021-01.csv"
zone_file = "/Users/krafton/project/personal/spark-airflow-hands-on/data/taxi+_zone_lookup.csv"

In [8]:
lines = sc.textFile(f"file:///{trip_files}")
header = lines.first()
filtered_lines = lines.filter(lambda row:row != header)

In [9]:
print(type(lines))
print(type(header))
print(type(filtered_lines))

<class 'pyspark.rdd.RDD'>
<class 'str'>
<class 'pyspark.rdd.PipelinedRDD'>


In [16]:
rdd = sc.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면",  "라면", "우동", "라면"])
type(rdd)

pyspark.rdd.RDD

In [17]:
rdd.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [18]:
rdd.take(1)

['짜장면']

# RDD Basic Handling

In [31]:
rdd.count()

12

In [20]:
rdd.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [23]:
rdd.distinct().collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [34]:
rdd.map(lambda x: x+'tt').take(3)

['짜장면tt', '마라탕tt', '짬뽕tt']

In [35]:
rdd.filter(lambda x: x != '짜장면').take(3)

['마라탕', '짬뽕', '떡볶이']

In [36]:
rdd.flatMap(lambda x: x.split('장')).take(3)

['짜', '면', '마라탕']

In [62]:
rdd.reduce(lambda x,y: x+y)

'짜장면마라탕짬뽕떡볶이쌀국수짬뽕짜장면짜장면짜장면라면우동라면'

In [51]:
rdd_groupby = rdd.groupBy(lambda x: x[0])
type(rdd_groupby)

pyspark.rdd.PipelinedRDD

In [52]:
rdd_groupby.take(3)

/opt/anaconda3/envs/spark-airflow/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[('짜', <pyspark.resultiterable.ResultIterable at 0x7fb6f80792e0>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7fb6f8079790>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7fb6f8079310>)]

In [65]:
for k, v in rdd_groupby.collect():
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


# key-value RDD
- pair RDD 로 기존의 RDD와 크게 다르지 않다
- key-value 형태로 데이터프레임처럼 사용할 수 있다는 장점
- key-value 로 나뉘어져 .mapValues(), .reduceByKey() 등을 사용할 수 있음
- value를 다루는 경우 .map() 보다는 .mapValues() 를 사용하자
  - 내부적으로 .mapValues()를 사용하면 파티션을 그대로 유지하면서 values 만을 건드려서 효율적


In [85]:
res = rdd.map(lambda x: (x, 1))
res.collect()

[('짜장면', 1),
 ('마라탕', 1),
 ('짬뽕', 1),
 ('떡볶이', 1),
 ('쌀국수', 1),
 ('짬뽕', 1),
 ('짜장면', 1),
 ('짜장면', 1),
 ('짜장면', 1),
 ('라면', 1),
 ('우동', 1),
 ('라면', 1)]

In [97]:
print(res.keys().collect())
print(res.values().collect())

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [86]:
res.mapValues(lambda x: x+10).collect()

[('짜장면', 11),
 ('마라탕', 11),
 ('짬뽕', 11),
 ('떡볶이', 11),
 ('쌀국수', 11),
 ('짬뽕', 11),
 ('짜장면', 11),
 ('짜장면', 11),
 ('짜장면', 11),
 ('라면', 11),
 ('우동', 11),
 ('라면', 11)]

In [87]:
res.reduceByKey(lambda x,y: x+y).collect()

[('짜장면', 4),
 ('마라탕', 1),
 ('짬뽕', 2),
 ('떡볶이', 1),
 ('쌀국수', 1),
 ('라면', 2),
 ('우동', 1)]

In [25]:
### foreach 는 rdd element 마다 해당 element 를 가지고 있는 executor 단에서 실행됨
### 여기서는 local에서 master 와 executor 를 둘다 가지고 있어 print 구문이 보이지만,
### 분산 환경에서 실행하면 print 구문이 보이지 않는다 (executor node 에 들어가야 보임)
### 이를 활용하여 executor node 에 pip install 을 보내는 방식으로 사용할 수도 있다
rdd.foreach(lambda x: print(x))

짜장면
마라탕
짬뽕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


# map, reduce 기능
- Spark는 분산된 환경에서 데이터 병렬 모델을 구현해 추상화 시켜준다
- map - reduce 의 병렬처리 질의문을 분산 환경에서도 일반적인 병렬 처리를 하듯이 추상화시켜준 것이다
  - 노드간 통신 같은 분산 환경에서 고려해야 하는 부분들은 스파크 내부 단에서 자동으로 처리해줌
- 그렇지만 성능 개선을 위해 분산 환경에서 병렬 처리하는 뒷단의 과정을 고려하여 코딩을 해야 한다
  - wide transformation (partition을 재조합 하는 과정) 이 비싼 과정임을 염두하자
  - .reduceByKey() 이전에 .filter()를 해주어 네트워크 통신 비용을 줄이도록 하자
- RDD 에서 고려해야 하는 이러한 성능 최적화가 DataFrame 에서는 내부 엔진 (Catalyst Optimizer)에서 자동으로 해준다
  - RDD는 row level 이다. 학습을 위해 그리고 정말 특수한 경우의 최적화를 위해 사용하고,
  - 가능하면 DF를 사용하자!

### 1) which one is better?
- paired_rdd.map().reduceByKey().filter().take(100)
- paired_rdd.map().filter().reduceByKey().take(100)

### 2) Answer
- reduceByKey() 는 shuffling 하는 함수이다. 즉, 비용이 비싼 함수!
- shuffling 하기 전에 filter() 를 해서 데이터를 줄여 네트워크 통신 비용을 줄이는 것이 성능 개선에 유리

### 3) narrow - wide transformation
- narrow: filter(), map(), flatMap(), sample(), union()
- wide: shuffling, join, reduceByKey(), groupByKey()

In [88]:
rdd.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [94]:
rdd.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[0]!='짜장면').collect()

/opt/anaconda3/envs/spark-airflow/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[('마라탕', 1), ('짬뽕', 2), ('떡볶이', 1), ('쌀국수', 1), ('라면', 2), ('우동', 1)]

In [95]:
rdd.map(lambda x: (x, 1)).filter(lambda x: x[0]!='짜장면').reduceByKey(lambda x,y: x+y).collect()

[('마라탕', 1), ('짬뽕', 2), ('떡볶이', 1), ('쌀국수', 1), ('라면', 2), ('우동', 1)]

# 파티셔닝 기능
- 노드에서 locally 연산을 최대한 수행하게끔 파티셔닝을 효율적으로 하면 성능개선에 유리함
- 분산 환경에서 데이터를 최대한 균일하게 퍼트려서 부하를 조절하는 것도 파티셔닝의 역할
- shuffling, 파티셔닝을 재조합하는 연산 이후에는 caching 하여 해당 연산이 반복되지 않도록 하자!
  - join, reduceByKey 같은 연산이 shuffling을 일으키며 비용이 비싼 연산이다.
  - 한번만 쓰는 경우라면 굳이 caching 하지 않는게 유리 -> 캐싱하는데에 시간이 걸림
  - 반복적으로 사용한다면 꼭 캐싱하자
- 파티셔닝 종류
  - Hash Partitioning
  - Range Partitioning

In [112]:
pairs = sc.parallelize([1, 2, 3, 4, 2, 4, 1]).map(lambda x: (x, x))
pairs.collect()

[(1, 1), (2, 2), (3, 3), (4, 4), (2, 2), (4, 4), (1, 1)]

In [114]:
pairs.partitionBy(2).glom().collect()

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

In [115]:
pairs.partitionBy(2, lambda x: x%2).glom().collect()

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

# cache - persist
- 스파크는 lazy-evaluatoin 이다
  - transformation 을 논리적으로 가지고 있다가
  - action 을 할 때에 그제서야 데이터를 memory로 옮기고 연산을 수행하여 결과를 낸다
  - 그리고 나서 memory 에 저장된 데이터를 다시 풀어준다
- 반복적으로 연산을 수행하는 데이터의 경우 memory 에 캐싱하여 사용하는 것이 효율적이다
  - .cache()
  - 메모리 할당을 풀어주려면 -> .unpersist()

In [108]:
rdd.cache()

ParallelCollectionRDD[7] at readRDDFromFile at PythonRDD.scala:274

In [109]:
rdd.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [110]:
rdd.unpersist()

ParallelCollectionRDD[7] at readRDDFromFile at PythonRDD.scala:274

In [111]:
rdd.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']